In [1]:
import csv
import json
import torch
from torch_geometric.data import DataLoader

In [2]:
data_list = torch.load('../gnn4rcpsp/data_list.tch')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data_loader = DataLoader(data_list, batch_size=32, shuffle=False)
bench_id = 0
task_durs_per_instance = []
for batch_idx, data_batch in enumerate(data_loader):
    data_batch.to(device)
    for data in data_batch.to_data_list():
        task_durs_per_instance.append(data.dur.data.cpu().detach().numpy().tolist())
        bench_id += 1

/home/aws_install/.venvs/gnn4rcpsp/lib64/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
with open('bench_list.json', 'r') as f:
    bench_list = json.load(f)

In [17]:
cpsat_json_dict = {}

with open('cpsat_solutions.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    instance_id = 0
    previous_row = None
    for row in reader:
        if int(row[0]) != instance_id:
            cpsat_json_dict.update({
                bench_list[instance_id]: {
                    'instance id': instance_id,
                    'sol': {
                        '_schedule_to_recompute': False,
                        'rcpsp_schedule': {
                            str(i+1): {
                                'start_time': int(s),
                                'end_time': int(s) + int(task_durs_per_instance[instance_id][i])
                            }
                        for i, s in enumerate(previous_row[4:])
                        },
                        'rcpsp_modes': [1 for _ in range(len(previous_row) - 4)],
                        'rcpsp_schedule_feasible': True
                    },
                    'status': 'SATISFIED',
                    'fit': {
                        'makespan': int(previous_row[3]),
                        'mean_resource_reserve': float('NaN')
                    }
                }
            })
            instance_id = int(row[0])
        previous_row = row

with open('cpsat_solutions.json', 'w') as jsonfile:
    json.dump(cpsat_json_dict, jsonfile, indent=2)